# 14-装饰器与闭包（Decorators and Closures）

大纲：

- 装饰器的基础知识
- Python何时执行装饰器
- 注册装饰器
- 变量作用域规则
- 闭包
- nonlocal声明
- 实现一个简单的装饰器
- 习题 

## 装饰器的基础知识

装饰器是一个可调用的对象，其参数是另一个函数（被装饰的函数）。装饰器可能会处理被装饰的函数，然后将其返回，或者将其替换为另一个函数或可调用对象。

In [6]:
# 定义一个装饰器
def deco(func):
    def inner():
        print('running inner()')
        
    # 使用内部函数替换被装饰的函数
    return inner

In [5]:
# 使用装饰器装饰另一个函数
@deco
def target():
    print('running target')
target()

running inner()


In [4]:
# 上面的代码等价于下面
def target():
    print('running target')
    
target = deco(target)
target()

running inner()


## Python何时执行装饰器

装饰器的一个关键性质是，它们在被装饰的函数定义之后立即运行。这通常是在导入时（例如，当Python 加载模块时）

In [9]:
# tag::REGISTRATION[]

registry = []  # <1>

def register(func):  # <2>
    print(f'running register({func})')  # <3>
    registry.append(func)  # <4>
    return func  # <5>

@register  # <6>
def f1():
    print('running f1()')

@register
def f2():
    print('running f2()')

def f3():  # <7>
    print('running f3()')

def main():  # <8>
    print('running main()')
    print('registry ->', registry)
    f1()
    f2()
    f3()

if __name__ == '__main__':
    main()  # <9>

# end::REGISTRATION[]

running register(<function f1 at 0x00000207AB132A20>)
running register(<function f2 at 0x00000207AB132DE0>)
running main()
registry -> [<function f1 at 0x00000207AB132A20>, <function f2 at 0x00000207AB132DE0>]
running f1()
running f2()
running f3()


1. registry列表用来存储被注册的函数
2. register装饰器用来注册函数，参数是被注册的函数
3. 打印被装饰的函数
4. 将func存入registry列表
5. 返回被装饰的函数
6. 使用@register装饰器注册函数f1和f2
7. 没有装饰f3
8. main函数打印registry列表，然后调用f1、f2和f3
9. 只有当前文件被执行时，才会调用main()函数

## 注册装饰器

考虑到装饰器在真实代码中的常用方式，示例有两处不寻常的地方。
- 示例中装饰器函数与被装饰的函数在同一个模块中定义。实际情况是，装饰器通常在一个模块中定义，然后再应用到其他模块中的函数上。
- register 装饰器返回的函数与通过参数传入的函数相同。实际上，大多数装饰器会在内部定义一个函数，然后将其返回。

## 变量作用域规则

In [ ]:
def f1(a):
    print(a)
    print(b)

In [ ]:
# 定义全局变量b
b = 6

def f2(a):
    # 打印局部变量a
    print(a)
    # 这里会发生什么？
    print(b)
    # 定义局部变量b
    b=9    
    
f2(3)

In [2]:
# 定义全局变量b
b = 6

def f3(a):
    # 打印局部变量a
    print(a)
    
    global b
    print(b)
    # 给全局变量赋值
    b=9    
    
f3(3)
# 打印全局变量b
print(b)

3
6
9


In [3]:
# dis模块可以反汇编python函数得到字节码
def f1(a):
    print(a)
    print(b)
    
from dis import dis
dis(f1)

  2           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  3           8 LOAD_GLOBAL              0 (print)
             10 LOAD_GLOBAL              1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP
             16 LOAD_CONST               0 (None)
             18 RETURN_VALUE


In [4]:
# 定义全局变量b
b = 6

def f2(a):
    # 打印局部变量a
    print(a)
    # 这里会发生什么？
    print(b)
    # 定义局部变量b
    b=9   
    
from dis import dis
dis(f2)

  6           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  8           8 LOAD_GLOBAL              0 (print)
             10 LOAD_FAST                1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP

 10          16 LOAD_CONST               1 (9)
             18 STORE_FAST               1 (b)
             20 LOAD_CONST               0 (None)
             22 RETURN_VALUE


## 闭包

In [5]:
# 一个计算累加值的类
class Averager:

    def __init__(self):
        self.series = []

    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total/len(self.series)
    
avg = Averager()

print(avg(10))
print(avg(11))
print(avg(12))

10.0
10.5
11.0


In [6]:
# 计算累加值的函数式实现

def make_averager():
    series = []
    
    def averager(new_value):
        # series = [1, 2, 3]
        series.append(new_value)
        total = sum(series)
        return total / len(series)

    return averager

avg2 = make_averager()
print(avg2(10))
print(avg2(11))
print(avg2(12))

10.0
10.5
11.0


![closure](./img/2024-05-06-11-30-33.png)

In [8]:
# 查看局部变量
avg2.__code__.co_varnames

('new_value', 'total')

In [10]:
# 查看自由变量
avg2.__code__.co_freevars

('series',)

In [13]:
# 查看闭包对象
print(avg2.__closure__)

# 查看闭包对象保存的数据
print(avg2.__closure__[0].cell_contents)

(<cell at 0x000002A70D6EDA50: list object at 0x000002A70DA34EC0>,)
[10, 11, 12]


## nonlocal声明

前面实现 make_averager 函数的方法效率不高。在示例 9-8 中，我们把所有值存储在历史数列中，然后在每次调用 averager 时使用 sum 求和。更好的实现方式是，只存储目前的总值和项数，根据这两个数计算平均值。

In [ ]:
def make_averager2():
    count = 0
    total = 0
    def averager(new_value):
        # 因为对counter赋值，counter被当作局部变量
        count += 1
        total += new_value
        return total / count
    return averager

avg3 = make_averager2()
avg3(10)

In [18]:
def make_averager3():
    count = 0
    total = 0
    def averager(new_value):
        # nonlocal把变量标记为自由变量
        nonlocal count, total        
        count += 1
        total += new_value
        return total / count
    return averager

avg4 = make_averager3()
avg4(10)

10.0

变量查找逻辑：

- 如果是 global x 声明，则 x 来自模块全局作用域，并赋予那个作用域中 x 的值。 
- 如果是 nonlocal x 声明，则 x 来自最近一个定义它的外层函数，并赋予那个函数中局部变量 x 的值。
- 如果 x 是参数，或者在函数主体中赋了值，那么 x 就是局部变量。
- 如果引用了 x，但是没有赋值也不是参数，则遵循以下规则。
  - 在外层函数主体的局部作用域（非局部作用域）内查找 x。
  - 如果在外层作用域内未找到，则从模块全局作用域内读取。
  - 如果在模块全局作用域内未找到，则从 __builtins__.__dict__ 中读取。

## 实现一个简单的装饰器

一个会显示函数运行时间的简单的装饰器

In [ ]:
import time

def clock(func):
    def clocked(*args):  # <1>
        t0 = time.perf_counter()
        result = func(*args)  # <2>
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print(f'[{elapsed:0.8f}s] {name}({arg_str}) -> {result!r}')
        return result
    return clocked  # <3>

## 缩短数值的过滤器(Number Shortening Filter)

难度：6kyu

在这个kata中，我们将创建一个函数，它返回另一个缩短长数字的函数。给定一个初始值数组替换给定基数的 X 次方。如果返回函数的输入不是数字字符串，则应将输入本身作为字符串返回。

例子：

```python
# shorten_number接受的输入是一个后缀列表，和一个基数，返回一个函数
filter1 = shorten_number(['','k','m'],1000)

# filter是一个函数，它接受一个数字字符串并返回一个数字字符串
filter1('234324') == '234k'
filter1('98234324') == '98m'
filter1([1,2,3]) == '[1,2,3]'

filter2 = shorten_number(['B','KB','MB','GB'],1024)
filter2('32') == '32B'
filter2('2100') == '2KB'
filter2('pippi') == 'pippi'
```

代码提交地址：
<https://www.codewars.com/kata/56b4af8ac6167012ec00006f>

按照下面的模式来编写自己的高阶函数：

- 定义一个外部的函数
- 在外部函数内部定义一个内部的函数
- 外部函数最后返回内部定义的函数

In [1]:
# 定义的外部函数
def shorten_number(suffixes, base):
    
    # 定义一个内部函数
    def my_filter(number):
        print(suffixes)
        print(base)
        # 在函数内部可以使用外部的变量suffixes，base
        return number     

    # 返回值是一个函数
    return my_filter

my_fun = shorten_number(['','k','m'],1000)
my_fun('234234')

['', 'k', 'm']
1000


'234234'

In [2]:
def shorten_number(suffixes, base):
    
    # 定义一个函数
    def my_filter(data):
        try:
            # 将函数输入转换为整数
            number = int(data)
            
        # 如果输入的数据不能转换为整数，直接转换为str返回
        except (TypeError, ValueError):
            return str(data)
        
        # 输入的number可以转换为整数
        else:
            # i用来跟踪suffixes列表的索引
            i = 0
            
            # 每次循环将输入的数字除以base，索引i+1
            # 如果除以base等于0或者索引等于len(suffixes)-1，结束循环
            while number//base > 0 and i < len(suffixes)-1:
                number //= base
                i += 1
            return str(number) + suffixes[i]     

    # 返回值是一个函数
    return my_filter

filter1 = shorten_number(['','k','m'],1000)
print(filter1('234324'))  # == '234k'
print(filter1('98234324')) # == '98m'
print(filter1([1,2,3])) # == '[1,2,3]'

filter2 = shorten_number(['B','KB','MB','GB', 'TB'],1024)
print(filter2('32')) # == '32B'
print(filter2('2100'))  # == '2KB';
print(filter2('2100000000000000000000'))  # == '2KB';
print(filter2('pippi')) # == 'pippi'

234k
98m
[1, 2, 3]
32B
2KB
1909938873TB
pippi
